In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 
import pandas as pd
import json
import requests
from pyquery import PyQuery

In [2]:
def get_links(page_num):
    # Extraction of all links till june 2020
    service = Service(ChromeDriverManager().install())


    #driver = webdriver.Chrome(service=service)
    browser = webdriver.Chrome(service=service)
    urls = []
    counter = 2
    articles_link = ["Links"]
    # since the tops category has 7 pages, where link to each follows a specific pattern identified above, we can create links to them as following:
    urls.append('https://www.altnews.in/topics/news/')
    for i in range(2, page_num):
        urls.append( 'https://www.altnews.in/topics/news/page/' + str(counter) + '/')
        counter += 1
    # extracting links for products in each page
    for url in urls:
        # open the url
        browser.get(url)
        # purposeful wait time to allow the website to get fully loaded
        time.sleep(4)
        # get page content
        content = browser.page_source
        soup = BeautifulSoup(content, "lxml")
        art_links = []
        # extract all the anchor i.e., <a> elements with “thumb-link” class from the page
        data_links = soup.find_all("a", {"aria-hidden": "true"})
        # from each <a> element, extract the URL
        for i in data_links:
            art_links.append(i['href'])

        articles_link.extend(art_links)
        # purposeful wait time to avoid sending requests in quick succession
        time.sleep(10)
    browser.quit()
    np.savetxt("Altnews_english_links_june2020.csv",
        articles_link,
        delimiter =", ",
        fmt ='% s')
    return

In [3]:
def get_content(dt):
    for url in dt["Links"]:
        #url = "https://www.altnews.in/modis-speech-mea-website-not-modis-speech-davos/"
        response = requests.get(str(url), stream=True)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tags = soup.find_all('meta')
            meta_features = {}
            for tag in meta_tags:
                if tag.get('property'):
                    meta_features[tag['property']] = tag.get('content')
            # title
            title.append(meta_features['og:title'])

            # Published_date
            json_data = json.loads(soup.find(type="application/ld+json").string)
            Published_date.append(json_data['@graph'][2]['datePublished'])

            # thumbnail
            #thumbnail.append(json_data['@graph'][5]['thumbnailUrl'])

            # topic
            topic.append(json_data['@graph'][3]['itemListElement'][1]['item'][36:-1])

            # Content
            data = soup.find_all('div', {'class':"entry-content e-c m-e-c guten-content", 'data-role': 'article_content'})
            if data:
                extracted_content = []
                for element in data:
                    for el in element.find_all(['p', 'blockquote']):
                        extracted_content.append(el.get_text())

                full_content = '\n'.join(extracted_content)

            Content.append(full_content)

            # links_in_text
            link = []
            liss = data[0].find_all('a')
            for i in liss[:-1]:
                if i.has_attr('href'):
                    link.append(i['href'])

            links_in_text.append(link)

            Linnnks.append(url)
    # save
    df = pd.DataFrame({'link':Linnnks, 'title': title, 'publish_date': Published_date, 'content': Content, 'Links_in_text':links_in_text, 'topic':topic})
    df.to_csv('Altnews_hindi_june2020.csv')
    return

In [5]:
if __name__ == "__main__":
    title = []
    Published_date = []
    Content = []
    video = []
    thumbnail = []
    links_in_text = []
    topic = []
    Linnnks = []
    get_links(34)
    dataframe = pd.read_csv("Altnews_english_links_june2020.csv")
    get_content(dataframe)